<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=87bb15f5b124c714a37d42bf8f662a194448107448124a72776f243d308d4b98
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-09 10:40:39
-------------------
qualified stocks: 90
with latest results: 27
still star stocks: 16


-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  5.27 L
Current:  1.40 C
-------------------
Today PnL: 30.13 K (0.22%)
Current PnL: -17.85 L (-12.0%)
CY Booked + Current PnL: -7.99 L (-5.37%)
-------------------
Total profit:  2.79 L
Total loss:  -20.64 L
-------------------
Total Booked + Current PnL: 18.99 L (15.61%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.04%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 79.14 L (56.48%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.83%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,DIXON,18940.15,-28.42,69.0,H-MC,8.48,143208.0,28272.0,8306.0,-0.58,24.60,5.80,31.83,81.0,3.40,1.06,44.40,X40N,ATH,IT
5,APOLLOHOSP,8285.00,-14.19,58.0,H-LC,10.76,171413.0,15236.0,10850.0,0.06,9.76,6.33,16.71,22.0,1.40,1.27,28.89,X40N,BTT,HEALTHCARE
80,TTKPRESTIG,770.00,105.34,63.0,M-SC,5.88,88458.0,-12319.0,12411.0,0.45,-12.22,14.03,0.09,245.0,-0.99,0.66,16.21,OX40N,NTT,DURABLES
65,SBIN,863.00,-13.25,46.0,M-LC,4.28,213749.0,12270.0,14941.0,-0.27,6.09,6.99,13.51,60.0,0.82,1.59,19.45,XY25,NTT,BANKS
34,HEROMOTOCO,5949.07,2.71,80.0,H-MC,22.93,190505.0,39592.0,17717.0,0.00,26.23,9.30,37.97,93.0,2.23,1.41,58.18,AR,ATH,AUTO


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,41.94,62.0,M-MC,8.02,223981.0,-981.0,168635.0,-1.05,-0.44,75.29,74.53,192.0,-0.01,1.66,31.68,XY24,BTT,STEEL
4,ANGELONE,3033.00,8.16,40.0,H-SC,6.49,191560.0,554.0,57142.0,-0.21,0.29,29.83,30.21,157.0,0.01,1.42,21.23,X40N,NTT,FINANCE
37,ICICIGI,2260.25,-21.04,39.0,H-MC,3.94,135642.0,-491.0,31618.0,-0.40,-0.36,23.31,22.86,91.0,-0.02,1.01,12.21,X40,ATH,INSURANCE
54,PGHH,17973.08,-31.08,50.0,H-MC,4.80,199365.0,-1455.0,70236.0,0.11,-0.72,35.23,34.25,80.0,-0.02,1.48,4.43,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HCLTECH,1943.91,-2.37,36.0,H-LC,10.18,223285.0,-18639.0,81901.0,1.37,-7.70,36.68,26.15,8.0,-0.23,1.66,5.51,X40,ATH,IT
86,VBL,671.64,-16.28,38.0,H-LC,6.66,301774.0,-14068.0,126745.0,-0.22,-4.45,42.00,35.67,5.0,-0.11,2.24,8.78,X40N,ATH,FMCG
77,TCS,4998.00,-28.69,38.0,H-LC,14.30,283786.0,-62170.0,186022.0,-0.96,-17.97,65.55,35.80,1.0,-0.33,2.10,0.53,X40,BTT,IT
46,JIOFIN,387.00,-1.80,41.0,H-LC,13.89,217105.0,-301.0,53799.0,-0.31,-0.14,24.78,24.61,48.0,-0.01,1.61,54.60,XY24,BTT,FINANCE
59,RELIANCE,1533.00,-14.41,44.0,H-LC,6.79,213876.0,3510.0,25280.0,-0.54,1.67,11.82,13.68,37.0,0.14,1.59,18.47,XY25,NTT,REFINERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LTIM,7230.20,-6.94,47.0,H-LC,1.63,230895.0,-15256.0,94459.0,0.83,-6.20,40.91,32.18,16.0,-0.16,1.71,27.61,X200,ATH,IT
53,NESTLEIND,1377.00,-9.50,56.0,H-LC,2.75,278390.0,12964.0,43819.0,0.17,4.88,15.74,21.40,11.0,0.30,2.06,13.05,XY25,NTT,FMCG
44,ITC,452.00,-38.34,49.0,H-LC,3.24,199536.0,-602.0,21490.0,0.12,-0.30,10.77,10.44,4.0,-0.03,1.48,5.25,X40,NTT,FMCG
20,CIPLA,1795.00,-20.81,47.0,H-LC,5.78,211624.0,7124.0,34283.0,0.23,3.48,16.20,20.25,10.0,0.21,1.57,12.52,X40N,BTT,PHARMA
86,VBL,671.64,-16.28,38.0,H-LC,6.66,301774.0,-14068.0,126745.0,-0.22,-4.45,42.00,35.67,5.0,-0.11,2.24,8.78,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VAIBHAVGBL,521.0,58.13,67.0,H-SC,10.97,105623.0,-46368.0,120495.0,6.59,-30.51,114.08,48.77,125.0,-0.38,0.78,22.94,XR,NTT,JEWELLERY
81,UJJIVANSFB,60.0,100.70,67.0,H-SC,12.84,126981.0,-15498.0,35021.0,6.47,-10.88,27.58,13.70,163.0,-0.44,0.94,49.87,OX40N,NTT,BANKS
42,INFY,2275.0,-18.16,51.0,H-LC,10.58,318328.0,5440.0,166231.0,4.30,1.74,52.22,54.87,3.0,0.03,2.36,8.50,X40,BTT,IT
66,SFL,1287.0,45.45,65.0,M-SC,9.03,192146.0,-70072.0,139902.0,3.99,-26.72,72.81,26.63,229.0,-0.50,1.42,19.47,XY24,NTT,MISC
78,TITAGARH,1548.0,-6.70,57.0,H-SC,7.86,153081.0,-38934.0,117811.0,3.52,-20.28,76.96,41.08,158.0,-0.33,1.14,29.72,XY24,NTT,ENGINEERING


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3737.84,64.0,M-SC,7.79,119617.0,-27751.0,79569.0,-5.30,-18.83,66.52,35.16,236.0,-0.35,0.89,22.18,XY24,NTT,AUTO
67,SHALBY,327.00,1057.89,70.0,M-SC,2.84,157547.0,-23596.0,69069.0,-2.99,-13.03,43.84,25.10,235.0,-0.34,1.17,26.26,XY24,NTT,HEALTHCARE
22,COFFEEDAY,80.00,-37.63,61.0,L-SC,40.49,91104.0,-22445.0,62497.0,-2.79,-19.77,68.60,35.27,268.0,-0.36,0.68,122.98,XR,NTT,HOTELS
71,STARHEALTH,761.00,8.04,51.0,H-SC,3.73,160628.0,-39588.0,115620.0,-2.01,-19.77,71.98,37.97,144.0,-0.34,1.19,28.97,XY24,NTT,INSURANCE
52,NATIONALUM,247.44,-45.32,66.0,H-MC,1.72,109874.0,9370.0,21272.0,-1.54,9.32,19.36,30.49,79.0,0.44,0.81,44.75,MH,ATH,METALS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UJJIVANSFB,60.0,100.70,67.0,H-SC,12.84,126981.0,-15498.0,35021.0,6.47,-10.88,27.58,13.70,163.0,-0.44,0.94,49.87,OX40N,NTT,BANKS
80,TTKPRESTIG,770.0,105.34,63.0,M-SC,5.88,88458.0,-12319.0,12411.0,0.45,-12.22,14.03,0.09,245.0,-0.99,0.66,16.21,OX40N,NTT,DURABLES
47,KANSAINER,340.0,-67.42,64.0,H-SC,2.58,224730.0,-44937.0,81262.0,2.63,-16.66,36.16,13.47,138.0,-0.55,1.67,13.67,XY24,NTT,PAINTS
40,INDIGOPNTS,1408.0,114.84,50.0,M-SC,3.23,140938.0,-33621.0,33656.0,-0.44,-19.26,23.88,0.02,221.0,-1.00,1.05,22.59,OX40N,NTT,PAINTS
19,CERA,9475.0,-19.57,47.0,H-SC,2.40,114660.0,-29143.0,55885.0,-0.23,-20.27,48.74,18.60,149.0,-0.52,0.85,26.53,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TCS,4998.00,-28.69,38.0,H-LC,14.30,283786.0,-62170.0,186022.0,-0.96,-17.97,65.55,35.80,1.0,-0.33,2.10,0.53,X40,BTT,IT
42,INFY,2275.00,-18.16,51.0,H-LC,10.58,318328.0,5440.0,166231.0,4.30,1.74,52.22,54.87,3.0,0.03,2.36,8.50,X40,BTT,IT
44,ITC,452.00,-38.34,49.0,H-LC,3.24,199536.0,-602.0,21490.0,0.12,-0.30,10.77,10.44,4.0,-0.03,1.48,5.25,X40,NTT,FMCG
86,VBL,671.64,-16.28,38.0,H-LC,6.66,301774.0,-14068.0,126745.0,-0.22,-4.45,42.00,35.67,5.0,-0.11,2.24,8.78,X40N,ATH,FMCG
1,ABB,7934.00,-41.23,44.0,H-LC,12.77,245515.0,-16104.0,135328.0,0.13,-6.16,55.12,45.57,7.0,-0.12,1.82,4.47,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,2.71,80.0,H-MC,22.93,190505.0,39592.0,17717.0,0.00,26.23,9.3,37.97,93.0,2.23,1.41,58.18,AR,ATH,AUTO
26,DIXON,18940.15,-28.42,69.0,H-MC,8.48,143208.0,28272.0,8306.0,-0.58,24.60,5.8,31.83,81.0,3.40,1.06,44.40,X40N,ATH,IT
27,DMART,5201.00,-6.54,64.0,H-LC,16.58,233382.0,48901.0,21471.0,1.03,26.51,9.2,38.14,38.0,2.28,1.73,39.92,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,2326.38,8.87,52.0,H-MC,3.35,187800.0,23130.0,44847.0,-0.33,14.05,23.88,41.28,89.0,0.52,1.39,23.15,X40N,ATH,AC
4,ANGELONE,3033.00,8.16,40.0,H-SC,6.49,191560.0,554.0,57142.0,-0.21,0.29,29.83,30.21,157.0,0.01,1.42,21.23,X40N,NTT,FINANCE
34,HEROMOTOCO,5949.07,2.71,80.0,H-MC,22.93,190505.0,39592.0,17717.0,0.00,26.23,9.30,37.97,93.0,2.23,1.41,58.18,AR,ATH,AUTO
87,VOLTAS,1918.49,1.02,59.0,H-MC,3.48,211965.0,20223.0,75799.0,-0.43,10.55,35.76,50.08,99.0,0.27,1.57,18.06,XY25,ATH,AC
24,DABUR,735.00,-2.94,60.0,H-MC,5.13,211282.0,14254.0,75364.0,-0.81,7.23,35.67,45.49,102.0,0.19,1.57,21.22,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,2.71,80.0,H-MC,22.93,190505.0,39592.0,17717.0,0.00,26.23,9.30,37.97,93.0,2.23,1.41,58.18,AR,ATH,AUTO
52,NATIONALUM,247.44,-45.32,66.0,H-MC,1.72,109874.0,9370.0,21272.0,-1.54,9.32,19.36,30.49,79.0,0.44,0.81,44.75,MH,ATH,METALS
88,WHIRLPOOL,2270.00,-39.15,64.0,M-SC,3.10,102938.0,11440.0,67311.0,0.70,12.50,65.39,86.07,223.0,0.17,0.76,49.48,XR,NTT,DURABLES
39,INDIAMART,4850.92,-50.91,50.0,H-SC,12.09,136846.0,13510.0,120247.0,0.29,10.95,87.87,108.45,119.0,0.11,1.01,35.76,AR,ATH,MISC
89,WIPRO,420.00,-12.43,50.0,M-LC,6.76,154132.0,3187.0,106274.0,2.51,2.11,68.95,72.51,53.0,0.03,1.14,8.26,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6716.67,52.0,L-SC,6.20,74814.0,-18996.0,95612.0,1.28,-20.25,127.80,81.67,269.0,-0.20,0.55,44.88,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-6.19,41.0,H-SC,14.99,87342.0,-13621.0,108156.0,-0.21,-13.49,123.83,93.63,148.0,-0.13,0.65,30.09,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-37.63,61.0,L-SC,40.49,91104.0,-22445.0,62497.0,-2.79,-19.77,68.60,35.27,268.0,-0.36,0.68,122.98,XR,NTT,HOTELS
51,MASFIN,398.61,-16.44,58.0,H-SC,12.94,94770.0,-3210.0,24811.0,1.79,-3.28,26.18,22.05,152.0,-0.13,0.70,37.76,XR,ATH,FINANCE
85,VALIANTORG,838.00,-332.69,63.0,H-SC,8.85,98826.0,-34460.0,119056.0,2.01,-25.85,120.47,63.47,139.0,-0.29,0.73,65.66,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-37.63,61.0,L-SC,40.49,91104.0,-22445.0,62497.0,-2.79,-19.77,68.60,35.27,268.0,-0.36,0.68,122.98,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,2.71,80.0,H-MC,22.93,190505.0,39592.0,17717.0,0.00,26.23,9.30,37.97,93.0,2.23,1.41,58.18,AR,ATH,AUTO
8,ATULAUTO,844.00,3737.84,64.0,M-SC,7.79,119617.0,-27751.0,79569.0,-5.30,-18.83,66.52,35.16,236.0,-0.35,0.89,22.18,XY24,NTT,AUTO
63,SAMMAANCAP,326.00,-171.59,64.0,M-SC,4.59,81984.0,-20226.0,113613.0,-0.78,-19.79,138.58,91.37,208.0,-0.18,0.61,33.09,XY25,NTT,FINANCE
27,DMART,5201.00,-6.54,64.0,H-LC,16.58,233382.0,48901.0,21471.0,1.03,26.51,9.20,38.14,38.0,2.28,1.73,39.92,X40N,NTT,FMCG


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.43
1,25,42.91
2,50,73.08


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.22
LC    35.48
MC    25.27
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.59
X40N     15.37
X40      15.31
XY25      9.86
AR        9.36
XR        9.13
OX40N     7.46
MH        1.71
X200      1.71
X5K       1.40
SR        1.07
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.28
H-SC    23.95
H-MC    22.29
M-SC    13.66
M-LC     7.14
M-MC     2.66
L-SC     1.61
L-LC     1.06
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.53,-1.06,35.74
IT,13.28,-15.20,72.32
FINANCE,8.71,-18.87,61.62
BANKS,7.36,-14.96,63.78
PAINTS,5.99,-14.56,40.20
MISC,5.80,-17.47,84.80
ELECTRICAL,5.58,-9.27,69.49
AUTO,4.81,-4.22,49.55
AC,3.68,2.50,30.52


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2746504.0,22
AR,1215958.0,10
XR,1061088.0,13
X40,902256.0,10
X40N,606806.0,12
OX40N,453225.0,10
XY25,421471.0,7
SR,198892.0,2
X5K,128615.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2790367.0,28
H-LC,1440581.0,16
M-SC,1405611.0,17
H-MC,1197882.0,17
M-LC,416704.0,5
M-MC,318396.0,2
L-SC,243479.0,3
L-MC,59821.0,1
L-LC,40685.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,923762.0,6
M-SC,XY24,819562.0,7
H-SC,AR,659654.0,5
H-LC,X40,605908.0,5
H-SC,XR,501924.0,6
H-MC,XY24,464111.0,4
H-LC,AR,378726.0,2
M-MC,XY24,318396.0,2
H-SC,X40N,249508.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
